In [1]:
!pip install tensorflow-gpu==2.7.0 opencv-python tensorflow_datasets matplotlib pycocotools;

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import cv2
import os
import tensorflow_datasets as tfds
import datetime
from coco import COCO
from coco_labels_paper import labels as coco_labels
import shutil
from tensorflow.keras.utils import to_categorical
import matplotlib as plt
from tensorflow import keras

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
BATCH_SIZE = 16
LAST_DENSE = 1024
SOFTMAX_OUT = True
FROM_LOGITS = False
LEARNING_RATE = 0.0001
INFO = "-"

#DATA_ROOT = "data/coco_onehot_data"
DATA_ROOT = "//bosqmode/koodit/Oppari/data/coco_onehot_data"
CONFIG_STRING = "batch_size: {0} | last_dense: {1} | softmax: {2} | from_logits: {3} | lr: {4} | info: {5}"

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.CategoricalAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [4]:
class DistanceLayer(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def call(self, anchor, validation):
        return tf.math.abs(anchor - validation)

class ResNet34:
    def __init__(self, input_shape=(224,224,3)):
        self.input_shape = input_shape
        self.model = self.CreateFeatureExtractor()

    def IdentityBlock(self, input, filters):
        conv1 = Conv2D(filters, (3,3), padding="same")(input)
        batchnorm1 = BatchNormalization(axis=3)(conv1)
        relu1 = ReLU()(batchnorm1)

        conv2 = Conv2D(filters, (3,3), padding="same")(relu1)
        batchnorm2 = BatchNormalization(axis=3)(conv2)

        add = Add()([batchnorm2,input])
        relu2 = ReLU()(add)
        return relu2

    def ConvolutionBlock(self, input, filters):
        conv1 = Conv2D(filters, (3,3), padding="same", strides=(2,2))(input)
        batchnorm1 = BatchNormalization(axis=3)(conv1)
        relu1 = ReLU()(batchnorm1)

        conv2 = Conv2D(filters, (3,3), padding="same")(relu1)
        batchnorm2 = BatchNormalization(axis=3)(conv2)

        linear_proj = Conv2D(filters, (1,1), strides=(2,2))(input)

        add = Add()([batchnorm2, linear_proj])
        relu2 = ReLU()(add)
        return relu2

    def CreateFeatureExtractor(self):
        input = Input(shape=self.input_shape)
        x = ZeroPadding2D((3,3))(input)
        x = Conv2D(64, (7,7), strides=(2,2))(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = MaxPool2D(pool_size=(3,3), strides=2, padding="same")(x)

        x = self.IdentityBlock(x, 64)
        x = self.IdentityBlock(x, 64)
        x = self.IdentityBlock(x, 64)
        x = self.ConvolutionBlock(x, 128)
        x = self.IdentityBlock(x, 128)
        x = self.IdentityBlock(x, 128)
        x = self.IdentityBlock(x, 128)
        x = self.ConvolutionBlock(x, 256)
        x = self.IdentityBlock(x, 256)
        x = self.IdentityBlock(x, 256)
        x = self.IdentityBlock(x, 256)
        x = self.IdentityBlock(x, 256)
        x = self.IdentityBlock(x, 256)
        x = self.ConvolutionBlock(x, 512)
        x = self.IdentityBlock(x, 512)
        x = self.IdentityBlock(x, 512)
        x = GlobalAveragePooling2D()(x)
        x = Flatten()(x)
        x = Dense(LAST_DENSE, activation="relu")(x)
        return Model(inputs=[input], outputs=[x], name='embedding')


def SiameseNetwork(input_shape, embedding):
    anchor = Input(name='anchor', shape=input_shape)
    validation = Input(name='validation', shape=input_shape)

    distances = DistanceLayer()(embedding(anchor),embedding(validation))
    classifier = Dense(1, activation='sigmoid')(distances)
    siamese_network = Model(inputs=[anchor, validation], outputs=classifier)
    return siamese_network


feature_extractor = ResNet34()
siamese_network = SiameseNetwork((224,224,3), feature_extractor.model)
siamese_network.compile(optimizer=Adam(0.0001))
siamese_network.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 anchor (InputLayer)            [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation (InputLayer)        [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 1024)         21831936    ['anchor[0][0]',                 
                                                                  'validation[0][0]']         

In [5]:
positives = None
negatives = None

datasets = {}

for c in coco_labels:
    if len(os.listdir(f'{DATA_ROOT}/{c}')) > 0:
        files = tf.data.Dataset.list_files(f'{DATA_ROOT}/{c}/*', shuffle=True, seed=12345).take(-1)
        datasets[c] = files

In [6]:
from itertools import cycle

In [7]:
for key, val in datasets.items():
    print(key)
    files = val.take(1000)
    half = int(len(files)/2)
    a = files.take(half)
    b = files.skip(half).take(half)
    c = None

    iterator = cycle(datasets.keys())
    for i in iterator:
        if c is None:
            c = datasets[i].take(10)
        else:
            c = c.concatenate(datasets[i].take(10))

        if len(c) >= half:
            break
        
    print(f'{len(a)} : {len(b)} : {len(c)}')

    if positives is None:
        positives = tf.data.Dataset.zip((a, b, tf.data.Dataset.from_tensor_slices(tf.ones(len(a)))))
        negatives = tf.data.Dataset.zip((a, c, tf.data.Dataset.from_tensor_slices(tf.zeros(len(a)))))
    else:
        positives = positives.concatenate(tf.data.Dataset.zip((a, b, tf.data.Dataset.from_tensor_slices(tf.ones(len(a))))))
        negatives = negatives.concatenate(tf.data.Dataset.zip((a, c, tf.data.Dataset.from_tensor_slices(tf.zeros(len(a))))))

print(f'Positives: {len(positives)}')
print(f'Negatives: {len(negatives)}')

final_data = positives.concatenate(negatives)
print(f'Final data: {len(final_data)}')

person
500 : 500 : 500
bicycle
500 : 500 : 500
car
500 : 500 : 500
motorcycle
500 : 500 : 500
airplane
500 : 500 : 500
bus
500 : 500 : 500
train
500 : 500 : 500
truck
500 : 500 : 500
boat
500 : 500 : 500
traffic light
500 : 500 : 500
fire hydrant
500 : 500 : 500
stop sign
500 : 500 : 500
parking meter
500 : 500 : 500
bench
500 : 500 : 500
bird
500 : 500 : 500
cat
500 : 500 : 500
dog
500 : 500 : 500
horse
500 : 500 : 500
sheep
500 : 500 : 500
cow
500 : 500 : 500
elephant
500 : 500 : 500
bear
500 : 500 : 500
zebra
500 : 500 : 500
giraffe
500 : 500 : 500
backpack
500 : 500 : 500
umbrella
500 : 500 : 500
handbag
500 : 500 : 500
tie
500 : 500 : 500
suitcase
500 : 500 : 500
frisbee
500 : 500 : 500
skis
500 : 500 : 500
snowboard
500 : 500 : 500
sports ball
500 : 500 : 500
kite
500 : 500 : 500
baseball bat
500 : 500 : 500
baseball glove
500 : 500 : 500
skateboard
500 : 500 : 500
surfboard
500 : 500 : 500
tennis racket
500 : 500 : 500
bottle
500 : 500 : 500
wine glass
500 : 500 : 500
cup
500 : 

In [8]:
def load_images(path):
    img = tf.io.read_file(path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224,224))
    img = img/255.0
    return img

def preprocess_twins(anchor, validation, label):
    return (load_images(anchor), load_images(validation), label)

In [11]:
processed_data = final_data.map(preprocess_twins)
processed_data = processed_data.cache()
processed_data = processed_data.shuffle(buffer_size=len(processed_data))

train_data = processed_data.take(round(len(processed_data)*.8))
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(8)

test_data = processed_data.skip(round(len(processed_data)*.8))
test_data = test_data.take(round(len(processed_data)*.2))
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(8)

In [ ]:
plt.imshow(t[1][0, :, :, :])

In [12]:
img_batch = next(iter(train_data))

fig = plt.pyplot.figure(figsize=(9,9))
axs = fig.subplots(2,2)

for i in range(2):
    axs[i, 0].imshow(img_batch[i][0])
    axs[i, 1].imshow(img_batch[i][1])

In [11]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir logs/siamese

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/siamese/{0}'.format(current_time)

summary_writer = tf.summary.create_file_writer(log_dir)
summary_writer.set_as_default()
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, update_freq=100)

Reusing TensorBoard on port 6006 (pid 35256), started 0:18:20 ago. (Use '!kill 35256' to kill it.)

In [13]:
binary_cross_loss = tf.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(1e-4)

In [16]:
@tf.function
def train_step(batch):
    
    # Record all of our operations 
    with tf.GradientTape() as tape:     
        # Get anchor and positive/negative image
        X = batch[:2]
        # Get label
        y = batch[2]
        
        # Forward pass
        yhat = siamese_network(X, training=True)
        # Calculate loss
        loss = binary_cross_loss(y, yhat)
        
    # Calculate gradients
    grad = tape.gradient(loss, siamese_network.trainable_variables)
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_network.trainable_variables))
        
    # Return loss
    return loss


In [18]:
from tensorflow.keras.metrics import Precision, Recall

def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Creating a metric object 
        r = Recall()
        p = Precision()
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            loss = train_step(batch)
            yhat = siamese_network.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())
        
        # Save checkpoints
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

In [19]:
EPOCHS = 50



train(train_data, EPOCHS)


 Epoch 1/50
1370/4000 [=========>....................] - ETA: 59:29

In [10]:
resnet.model.save_weights(os.path.join(weight_dir, weight_file.format("_onehot")))

In [12]:

resnet.model.evaluate(validation_ds, verbose=1)


14/14 [==============================] - 2s 151ms/step - loss: 2.2457 - accuracy: 0.6529


[2.2457149028778076, 0.6528735756874084]